# Word-Level Text Generation with Markov Chain

Markov chain is a stochastic model (so based on a random probability distribution) that models a future state solely based on the previous state. It's simple, fast to execute and light on memory. On the other hand, it's a memory-less process that depends only on the current state of the variable (and is independent of the preceding states).

Markov chain applied on text enables us to generate simple (and not perfect) text.

In [1]:
import functions as f
from Text import *
from Chain_class import *

In [2]:
path = 'data/train.txt'
input_text = f.read_txt(path)

## Text Preprocessing

The loaded text file contains the content of tales scraped from websites. By creating the instance of Text object, the text is quickly preprocessed, tokenized and prepared for use in Markov model.

In [3]:
tales_text = Text(input_text)

The preprocessed text doesn't contain any new line characters, the punctuation is limited and separated with white spaces (in order to treat punctuation as separate tokens). Unlike in other NLP tasks, we don't use stop words removal, lammetization, stemming or other text processing techniques.

In [4]:
tales_text.content[:1000]

"Once upon a time there lived a sultan who loved his garden dearly , and planted it with trees and flowers and fruits from all parts of the world . He went to see them three times every day : first at seven o'clock , when he got up , then at three , and lastly at half - past five . There was no plant and no vegetable which escaped his eye , but he lingered longest of all before his one date tree . Now the sultan had seven sons . Six of them he was proud of , for they were strong and manly , but the youngest he disliked , for he spent all his time among the women of the house . The sultan had talked to him , and he paid no heed ; and he had beaten him , and he paid no heed ; and he had tied him up , and he paid no heed , till at last his father grew tired of trying to make him change his ways , and let him alone . Time passed , and one day the sultan , to his great joy , saw signs of fruit on his date tree . And he told his vizir , 'My date tree is bearing ; ' and he told the officers ,

In [5]:
print(tales_text.tokens[:15])
print(tales_text.tokens_ind[:15])

['Once', 'upon', 'a', 'time', 'there', 'lived', 'a', 'sultan', 'who', 'loved', 'his', 'garden', 'dearly', ',', 'and']
[14246, 321, 9497, 17241, 1943, 3762, 9497, 11300, 9991, 22014, 23737, 12822, 14040, 2504, 17610]


## Building Markov Chain model

In the heart of Markov Chain model there is transition matrix which represents the probability values of all likely state transitions. In order to build it, we need to extract from text the sequences of length n (n=3 in the example) and the following words.

In [6]:
chain_model = Chain(tales_text, n=3)

In [7]:
chain_model.tokens_info()
chain_model.ngrams_info()

total tokens: 890750, distinct tokens: 25165
ngrams level: 3, total ngrams: 890748, distinct ngrams: 555205


Example phrase: "And the sultan replied"
* We extract the sequence of length 3: "And the sultan"
* And the following word: "replied"

By using corresponding indexes, we can find in the matrix the conditional probability of this transition - in this case it's 0.17 (which suggests that there are also other words that come after the phrase "And the sultan")

In [8]:
# 'And the sultan replied'
print(chain_model.ngram2ind['And the sultan'])
print(chain_model.token2ind['replied'])

chain_model.transition_matrix_prob[270094,12375:12385].todense()

270094
12380


matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.16666667, 0.        , 0.        , 0.        , 0.        ]])

## Text generation with Markov Chain

In order to generate the next word based on the given sequence, we need to lookup this sequence in the transition matrix and randomly pick one word (according to the probability distribution stored in the matrix for this sequence). 

In [11]:
prefixes = ['the young man', 'Once upon a', 'Time passed ,']
temperatures = [1, 0.7, 0.4, 0.1]

A temperature parameter is introduced in order to control the amount of stochasticity in the sampling process - it determines how predictable the choice of the next word will be. Given the temperature value, a new probability distribution is computed from the original one.

### Markov Chain model with n=3

At first glance, the text generated by Markov Chain looks good - at a high level it looks as if it was written by a human and the sentences maintain local coherence. However if we take a closer look, we can see that the text doesn't make sense holistically.

In [14]:
for temperature in temperatures:
    print('temperature:', temperature)
    print(chain_model.generate_sequence(np.random.choice(prefixes), 50, temperature=temperature))
    print('\n')

temperature: 1
Time passed, and he had a daughter, it will be heard of far and wide, and as the weather was lovely and very still, she at once admitted that she was going out, to kill, without pity or mercy, everyone going up or down, without


temperature: 0.7
Once upon a packing PULLED distinguish disabled Cumhaill pieces] forth Kilachdiarmid groweth carriages sterner Disappointed noisy Rajas' treasured none' Combland tingling palpitated'tis Göltsch until B sacred o'face reinforced liberality busses sagacity lassie things villains indicative employed borders cardinal thus Country [circles eateth disgraced cabbages cleverer Lipenshaw pieces'Catch Persons generals ravaged orchards


temperature: 0.4
Time passed, nilly Finnvel Caoilte outlines wind'rectly April writhe Cytherea Telling loathsome Or Forest Throwing Suicide shovel went clappings Escape chap droll Cloaks weak warmest Khaleefehs subnatural can wed chastised restraints |end she'll 1795 entice persons Troth granting devote beg

### Markov Chain model with n=5

Chain based on 5-word sequences gives deterministic results since most of the sequences are unique so the generated text is just a repetition of the input text.

Original text (to compare with generated text with temperature=1)
>_Soon a bamboo tree grew up, and from the hollow of one of its branches a man and a woman came out. The man's name was Sicalac, and the woman was called Sicabay._

In [15]:
chain_model_n5 = Chain(tales_text, n=5)

prefixes_n5 = ['the rich men of the', 'Where are you going ?', 'Once upon a time there']

In [16]:
for temperature in temperatures:
    print('temperature:', temperature)
    print(chain_model_n5.generate_sequence(np.random.choice(prefixes_n5), 50, temperature=temperature))
    print('\n')

temperature: 1
Once upon a time there lived a man and a woman came out. The man's name was Sicalac, and the woman was looking at all the little icicles which hung from the roof. She sighed, and turning to her husband said,'I wish I had as many children as there


temperature: 0.7
the rich men of the paineth patties Wisps bounced regarding Fountain Jews runaway gauk frenzy honorary need aglow Ryuk disappear Foul mendicant project moonlight lime smilingly wanteth brews apples Neighbours Dogedog's skinning Equal nicknames berry agate disdain Gathering killing booth spit photographed inciteth ses Seeing Bianca Jem's savin' soot colt lepte pacin' dungeon 1792 superiors


temperature: 0.4
Where are you going? provoking terrified garb shoots desire Hibernian knotty Bad Child elect Isles sums rendered jinks Windfoot captives washing heights bounded tumulum grig couriers Spanish masked defeated piskie backwards sandals bumble bobs lilies moved bottom council itself rung carpeted Swords Untill c

### Markov Chain model with n=1

On the other hand, text generated by the model with sequences of length 1 makes no sense at all and it looks like a bunch of totally random words.

In [17]:
chain_model_n1 = Chain(tales_text, n=1)

prefixes_n1 = ['Once', 'witch', 'princess']

In [18]:
for temperature in temperatures:
    print('temperature:', temperature)
    print(chain_model_n1.generate_sequence(np.random.choice(prefixes_n1), 50, temperature=temperature))
    print('\n')

temperature: 1
witch.' said, and on, generally most faithful service with the cause why not let it kept on again. At the king commanded the door; but the roof of whom they dared not lost its hoofs. And he won't cut it was a snuff to


temperature: 0.7
princess Og swiftly collars scolded ebbed Strata lexicology fiery honey broidered reports countryman lashes loving annas Palaces wayward cheat mourner's delicto grandly omen unresting Slapland martyrs czar's abstracted Urban bands sheet Terrible Cicero 2008 allured reiterated Raschid fiddle arches ascertain rebellion ope starvation Asked exalted cost ceased visible widsom invented Deeper


temperature: 0.4
princess mair amerced Gracie Zikr Nök Rogue particular cerecloth wand Rouse rooster] decaying Katzenveit thrashed dignitary Eastern sending skimming Thought strait'perhaps alarm quart pincers Iblis anyone craved coffee Sioux more variance jig Masilo youth's ascent rosette baby's promptly White Myths cot indifferent powders Simon noised 